In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import csv
from PIL import Image
import requests
import torch
import pandas as pd
import tarfile
import os
import warnings

In [ ]:
#Model currently runing
from transformers import Pix2StructForConditionalGeneration, Pix2StructProcessor
model = Pix2StructForConditionalGeneration.from_pretrained("google/pix2struct-screen2words-base").to("cuda")
processor = Pix2StructProcessor.from_pretrained("google/pix2struct-screen2words-base")

#USE model
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
embed = hub.load(module_url)
print("\nUSE model loaded. \n")

In [ ]:
#Dataset
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import csv
from PIL import Image
import requests
import torch
import pandas as pd
import tarfile
import os
import warnings
warnings.filterwarnings("ignore", message="Using `max_length`'s default")

torch.cuda.empty_cache()

#Model currently runing

   
#https://huggingface.co/google/pix2struct-screen2words-base#using-the-model
def pix2struct_half_gpu(image, temperature, beam_width): #Trained with screen2words(half precision)
    image = Image.open(image)
    from transformers import Pix2StructForConditionalGeneration, Pix2StructProcessor
    model = Pix2StructForConditionalGeneration.from_pretrained("google/pix2struct-screen2words-base", torch_dtype=torch.bfloat16).to("cuda")
    processor = Pix2StructProcessor.from_pretrained("google/pix2struct-screen2words-base")
    # image only
    inputs = processor(images=image, text="Mobile application screenshot", return_tensors="pt").to("cuda", torch.bfloat16)

    predictions = model.generate(**inputs, temperature = temperature, num_beams = beam_width, do_sample=True, num_return_sequences=5)
    #print(processor.decode(predictions[0], skip_special_tokens=True))

    model_name = "pix2struct_half_gpu"
    
    pred_step = []

    pred_step.append( str(processor.decode(predictions[0], skip_special_tokens=True)) )
    pred_step.append( str(processor.decode(predictions[1], skip_special_tokens=True)) )
    pred_step.append( str(processor.decode(predictions[2], skip_special_tokens=True)) )
    pred_step.append( str(processor.decode(predictions[3], skip_special_tokens=True)) )
    pred_step.append( str(processor.decode(predictions[4], skip_special_tokens=True)) )

  
    similarity_treshold = 0.6
    j = 0
    aux = 0
    sim_max = -1
    while (j < len(pred_step)):
        i = 0
        sim_average = 0
        while (i < len(pred_step)):
            if (i != j):
                embeddings = embed((pred_step[j], pred_step[i]))
                sim = cosine_similarity([embeddings[0]], [embeddings[1]])
                sim_average += sim
            i += 1
        sim_average = sim_average/i
        if sim_average > sim_max:
            sim_max = sim_average
            pred_max = pred_step[j]

        if (sim_average < similarity_treshold):
            pred_step[j] = ""
        else:
            aux = 1
            
        j += 1

    if aux == 0:
        pred_step[0] = pred_max
        pred_step[1] = str(sim_max)
    #print(pred_step)

    return str(pred_step), model_name

#https://huggingface.co/google/pix2struct-screen2words-base#using-the-model
def pix2struct_full_gpu(image, temperature, beam_width): #Trained with screen2words(full precision)

    image = Image.open(image)
    
    # image only
    inputs = processor(images=image, text="Mobile application screenshot", return_tensors="pt").to("cuda")

    predictions = model.generate(**inputs, temperature = temperature, num_beams = beam_width, do_sample=True, num_return_sequences=5)
    #predictions = model.generate(**inputs.to(torch.bfloat16), temperature = temperature, num_beams = beam_width, do_sample=True, num_return_sequences=5)
    #print(processor.decode(predictions[0], skip_special_tokens=True))

    model_name = "pix2struct_full_gpu"
    pred_step = []

    pred_step.append( str(processor.decode(predictions[0], skip_special_tokens=True)) )
    pred_step = str(pred_step)
    pred_step = pred_step.replace("'","")

    '''
    pred_step.append( str(processor.decode(predictions[1], skip_special_tokens=True)) )
    pred_step.append( str(processor.decode(predictions[2], skip_special_tokens=True)) )
    pred_step.append( str(processor.decode(predictions[3], skip_special_tokens=True)) )
    pred_step.append( str(processor.decode(predictions[4], skip_special_tokens=True)) )

  
    similarity_treshold = 0.6
    j = 0
    aux = 0
    sim_max = -1
    while (j < len(pred_step)):
        i = 0
        sim_average = 0
        while (i < len(pred_step)):
            if (i != j):
                embeddings = embed((pred_step[j], pred_step[i]))
                sim = cosine_similarity([embeddings[0]], [embeddings[1]])
                sim_average += sim
            i += 1
        sim_average = sim_average/i
        if sim_average > sim_max:
            sim_max = sim_average
            pred_max = pred_step[j]

        if (sim_average < similarity_treshold):
            pred_step[j] = ""
        else:
            aux = 1
            
        j += 1

    if aux == 0:
        pred_step[0] = pred_max
        pred_step[1] = str(sim_max)
    #print(pred_step)
    '''

    return pred_step, model_name



#initializing text comparing model and file
filename = "Annotations/screen_summaries.csv"
column_sum = "summary" 
column_id = "screenId"

image_count = 0

#models
models = [pix2struct_full_gpu] #pix2struct_full_gpu, pix2struct_half_gpu, blip, pix2stuct_full, pix2struct_prompt, bipin, GIT,  nlpconnect, donut
model_count = 0

#temperatures
temperatures = [0.49] #0.31,0.32,0.33,0.34,0.35,0.36,0.37,0.38,0.39 0.41,0.42,0.43,0.44,0.45,0.46,0.47,0.48,0.49,0.52,0.53,0.54,0.55
#beam_width
beam_widths = [1]

# Set the path to the .tar.gz file
tar_gz_path = 'Images/unique_uis.tar.gz'

# Open the .tar.gz file in read mode
tar = tarfile.open(tar_gz_path, 'r:gz')

# List the contents of the .tar.gz file
tar_contents = tar.getnames()

# Extract the contents of the .tar.gz file to a directory
extract_dir = 'Images'

similarity_treshold = 0.3

# Open a file for writing
with open('Results/Dataset_final_6.csv'  ,mode='w', newline='') as summary_file:
    writer_sum = csv.writer(summary_file)
    writer_sum.writerow(['ImageID','Description'])

    while (model_count < len(models)):
          temp_count = 0
          while (temp_count < len(temperatures)):
              beam_count = 0
              temp_sim_average = 0
              while (beam_count < len(beam_widths)):
                  model_average = 0
                  image_count =  132422 
                  processed_images = 0
                  while (image_count < len(tar_contents) ): #and processed_images < 200

                      if os.path.splitext(tar_contents[image_count])[1].lower() == '.jpg':
                          tar.extract(tar_contents[image_count], path=extract_dir)
                          image_id = tar_contents[image_count].split("/", 1)[1]
                          image_id = tar_contents[image_count].split(".", 1)[0]
                          image_id = image_id.split("/", 1)[1]

                          with open(filename, "r") as csvfile:
                              reader = csv.DictReader(csvfile)
                              
                              '''
                              #Checking if image is not part of the Screen2Words dataset
                              aux = 1
                              for row in reader:
                                  if (row[column_id] == image_id):
                                      aux = 0
                                      break

                              csvfile.seek(0)
                              if aux == 1:'''
                              processed_images +=1
                              argument = 'Images/' + tar_contents[image_count]
                              model_string, model_name = models[model_count](argument, temperatures[temp_count], beam_widths[beam_count])
                              writer_sum.writerow([image_id, model_string])
                              print(processed_images, "(",image_count+1,")", "\n", model_name, image_id, model_string)
                          os.remove('Images/' + tar_contents[image_count])
                      image_count += 1
                  beam_count += 1
              temp_count +=1
          model_count += 1

# Close the .tar.gz file
tar.close()
#132523

In [ ]:
#Testing Temperatures
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import csv
from PIL import Image
import requests
import torch
import pandas as pd
import tarfile
import os
import warnings
warnings.filterwarnings("ignore", message="Using `max_length`'s default")

torch.cuda.empty_cache()
#https://huggingface.co/docs/transformers/main/model_doc/git#transformers.GitForCausalLM.forward.example
def GIT (image, temperature, beam_width): #QA


    from transformers import AutoProcessor, AutoModelForCausalLM
    from huggingface_hub import hf_hub_download


    processor = AutoProcessor.from_pretrained("microsoft/git-base-textvqa")
    model = AutoModelForCausalLM.from_pretrained("microsoft/git-base-textvqa")
    #file_path = hf_hub_download(repo_id="nielsr/textvqa-sample", filename="bus.png", repo_type="dataset")
    image = Image.open(image).convert("RGB")

    pixel_values = processor(images=image, return_tensors="pt").pixel_values

    question = "What is the purpose of this mobile application?"

    input_ids = processor(text=question, add_special_tokens=False).input_ids
    input_ids = [processor.tokenizer.cls_token_id] + input_ids
    input_ids = torch.tensor(input_ids).unsqueeze(0)

    generated_ids = model.generate(pixel_values=pixel_values, input_ids=input_ids, max_length=50)
    #print(processor.batch_decode(generated_ids, skip_special_tokens=True))
    #['what does the front of the bus say at the top? special']

    model_name = "GIT"
    pred_step = str(processor.batch_decode(generated_ids, skip_special_tokens=True))
    pred_step = pred_step.split("?", 1)[1]
    pred_step = pred_step.split("'", 1)[0]

    return pred_step, model_name

#https://huggingface.co/bipin/image-caption-generator
def bipin (image, temperature, beam_width): #makes captions
    model_name = "bipin/image-caption-generator"

    # load model
    from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, AutoTokenizer
    model = VisionEncoderDecoderModel.from_pretrained(model_name)
    feature_extractor = ViTFeatureExtractor.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained("gpt2")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)



    img_name = image
    img = Image.open(img_name)
    if img.mode != 'RGB':
        img = img.convert(mode="RGB")

    pixel_values = feature_extractor(images=[img], return_tensors="pt").pixel_values
    pixel_values = pixel_values.to(device)

    max_length = 128
    num_beams = 5

    # get model prediction
    output_ids = model.generate(pixel_values, num_beams=num_beams, max_length=max_length,temperature=temperature)

    # decode the generated prediction
    preds = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    #print(preds)

    model_name = "bipin"
    pred_step = preds

    return pred_step, model_name


#https://huggingface.co/docs/transformers/main/en/model_doc/donut
def donut (image, temperature, beam_width): #QA model

    import re
    from transformers import DonutProcessor, VisionEncoderDecoderModel
    from datasets import load_dataset
    
    processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base-finetuned-docvqa")
    model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base-finetuned-docvqa", torch_dtype=torch.bfloat16).to("cuda")
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    image = Image.open(image)

    # prepare decoder inputs
    task_prompt = "<s_docvqa><s_question>{user_input}</s_question><s_answer>"
    question = "What is this application about?"
    prompt = task_prompt.replace("{user_input}", question)
    decoder_input_ids = processor.tokenizer(prompt, add_special_tokens=False, return_tensors="pt").input_ids

    pixel_values = processor(image, return_tensors="pt").pixel_values

    outputs = model.generate(
        pixel_values.to(device),
        decoder_input_ids=decoder_input_ids.to(device),
        max_length=model.decoder.config.max_position_embeddings,
        early_stopping=True,
        pad_token_id=processor.tokenizer.pad_token_id,
        eos_token_id=processor.tokenizer.eos_token_id,
        use_cache=True,
        num_beams=1,
        bad_words_ids=[[processor.tokenizer.unk_token_id]],
        return_dict_in_generate=True,
        temperature = temperature
    ).to("cuda")

    sequence = processor.batch_decode(outputs.sequences)[0]
    sequence = sequence.replace(processor.tokenizer.eos_token, "").replace(processor.tokenizer.pad_token, "")
    sequence = re.sub(r"<.*?>", "", sequence, count=1).strip()  # remove first task start token
    #print(processor.token2json(sequence))

    model_name = "donut"
    pred_step = str(processor.token2json(sequence))
    pred_step = pred_step.split(",", 1)[1]
    pred_step = pred_step.split(":", 1)[1]
    pred_step = pred_step.replace("'","")
    pred_step = pred_step.replace("}","")
    
    return str(pred_step), model_name

#https://huggingface.co/nlpconnect/vit-gpt2-image-captioning
def nlpconnect(image, temperature): #Normal image-to-text model
    from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer

    model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
    feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
    tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    max_length = 16
    num_beams = 4
    gen_kwargs = {"max_length": max_length, "num_beams": num_beams}

    def predict_step(image_paths):
        images = []
        for image_path in image_paths:
            i_image = Image.open(image_path)
            if i_image.mode != "RGB":
                i_image = i_image.convert(mode="RGB")

            images.append(i_image)

        pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
        pixel_values = pixel_values.to(device)

        output_ids = model.generate(pixel_values, **gen_kwargs, temperature=temperature)

        preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
        preds = [pred.strip() for pred in preds]
        return preds

    model_name = "nlpconnect"
    pred_step = str(predict_step([image]))
    pred_step = pred_step.split("'", 1)[1]
    pred_step = pred_step.split("'", 1)[0]

    return pred_step, model_name


#https://huggingface.co/google/pix2struct-screen2words-base#using-the-model
def pix2struct_half_gpu(image, temperature, beam_width): #Trained with screen2words(half precision)
    image = Image.open(image)


    from transformers import Pix2StructForConditionalGeneration, Pix2StructProcessor
    model = Pix2StructForConditionalGeneration.from_pretrained("google/pix2struct-screen2words-base", torch_dtype=torch.bfloat16).to("cuda")
    processor = Pix2StructProcessor.from_pretrained("google/pix2struct-screen2words-base")


    # image only
    inputs = processor(images=image, text="Mobile application screenshot", return_tensors="pt").to("cuda", torch.bfloat16)

    predictions = model.generate(**inputs, temperature = temperature, num_beams = beam_width, do_sample=True, num_return_sequences=5)
    #print(processor.decode(predictions[0], skip_special_tokens=True))

    model_name = "pix2struct_half_gpu"
    
    pred_step = []

    pred_step.append( str(processor.decode(predictions[0], skip_special_tokens=True)) )
    pred_step.append( str(processor.decode(predictions[1], skip_special_tokens=True)) )
    pred_step.append( str(processor.decode(predictions[2], skip_special_tokens=True)) )
    pred_step.append( str(processor.decode(predictions[3], skip_special_tokens=True)) )
    pred_step.append( str(processor.decode(predictions[4], skip_special_tokens=True)) )

  
    similarity_treshold = 0.6
    j = 0
    aux = 0
    sim_max = -1
    while (j < len(pred_step)):
        i = 0
        sim_average = 0
        while (i < len(pred_step)):
            if (i != j):
                embeddings = embed((pred_step[j], pred_step[i]))
                sim = cosine_similarity([embeddings[0]], [embeddings[1]])
                sim_average += sim
            i += 1
        sim_average = sim_average/i
        if sim_average > sim_max:
            sim_max = sim_average
            pred_max = pred_step[j]

        if (sim_average < similarity_treshold):
            pred_step[j] = ""
        else:
            aux = 1
            
        j += 1

    if aux == 0:
        pred_step[0] = pred_max
        pred_step[1] = str(sim_max)
    #print(pred_step)

    return str(pred_step), model_name

#https://huggingface.co/google/pix2struct-screen2words-base#using-the-model
def pix2struct_full_gpu(image, temperature, beam_width): #Trained with screen2words(full precision)

    image = Image.open(image)
    
    # image only
    inputs = processor(images=image, text="Mobile application screenshot", return_tensors="pt").to("cuda")

    predictions = model.generate(**inputs, temperature = temperature, num_beams = beam_width, do_sample=True, num_return_sequences=5)
    #predictions = model.generate(**inputs.to(torch.bfloat16), temperature = temperature, num_beams = beam_width, do_sample=True, num_return_sequences=5)
    #print(processor.decode(predictions[0], skip_special_tokens=True))

    model_name = "pix2struct_full_gpu"
    pred_step = []

    pred_step.append( str(processor.decode(predictions[0], skip_special_tokens=True)) )
    pred_step.append( str(processor.decode(predictions[1], skip_special_tokens=True)) )
    pred_step.append( str(processor.decode(predictions[2], skip_special_tokens=True)) )
    pred_step.append( str(processor.decode(predictions[3], skip_special_tokens=True)) )
    pred_step.append( str(processor.decode(predictions[4], skip_special_tokens=True)) )

  
    similarity_treshold = 0.6
    j = 0
    aux = 0
    sim_max = -1
    while (j < len(pred_step)):
        i = 0
        sim_average = 0
        while (i < len(pred_step)):
            if (i != j):
                embeddings = embed((pred_step[j], pred_step[i]))
                sim = cosine_similarity([embeddings[0]], [embeddings[1]])
                sim_average += sim
            i += 1
        sim_average = sim_average/i
        if sim_average > sim_max:
            sim_max = sim_average
            pred_max = pred_step[j]

        if (sim_average < similarity_treshold):
            pred_step[j] = ""
        else:
            aux = 1
            
        j += 1

    if aux == 0:
        pred_step[0] = pred_max
        pred_step[1] = str(sim_max)
    #print(pred_step)

    return str(pred_step), model_name



#initializing text comparing model and file
filename = "Annotations/screen_summaries.csv"
column_sum = "summary" 
column_id = "screenId"


#images
#images = ["1264"] #"1264", "1304", "3084", "11337", "13434", "15050", "19821", "21141", "23608", "25372", "25675", "27375", "27392", "29876", "31452", "32435", "34025", "35567", "35604", "35858"
image_count = 0

#models
models = [pix2struct_full_gpu] #pix2struct_full_gpu, pix2struct_half_gpu, blip, pix2stuct_full, pix2struct_prompt, bipin, GIT,  nlpconnect, donut
model_count = 0

#temperatures
temperatures = [0.49] #0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,1.25,1.5,  0.31,0.32,0.33,0.34,0.35,0.36,0.37,0.38,0.39 0.41,0.42,0.43,0.44,0.45,0.46,0.47,0.48,0.49,0.52,0.53,0.54,0.55
#beam_width
beam_widths = [1]

# Set the path to the .tar.gz file
tar_gz_path = 'Images/unique_uis.tar.gz'

# Open the .tar.gz file in read mode
tar = tarfile.open(tar_gz_path, 'r:gz')

# List the contents of the .tar.gz file
tar_contents = tar.getnames()

# Extract the contents of the .tar.gz file to a directory
extract_dir = 'Images'

similarity_treshold = 0.3

# Open a file for writing
with open('Results/Results_summary'+str(temperatures[0])+'-'+str(temperatures[-1])+'.csv', mode='w', newline='') as summary_file:
    writer_sum = csv.writer(summary_file)
    writer_sum.writerow(['Model', 'Temperature', 'Average Similarity'])

    with open('Results/Results'+str(temperatures[0])+'-'+str(temperatures[-1])+'.csv', mode='w', newline='') as file:

        # Create a writer object
        writer = csv.writer(file)

        # Write a row to the CSV file
        writer.writerow(['Model', 'ImageID','Description','Average Similarity', 'Temperature', 'Beam Width'])

        while (model_count < len(models)):
              temp_count = 0
              while (temp_count < len(temperatures)):
                  beam_count = 0
                  temp_sim_average = 0
                  while (beam_count < len(beam_widths)):
                      model_average = 0
                      image_count = 0
                      processed_images = 0
                      print(len(tar_contents))
                      while (image_count < len(tar_contents) and processed_images < 2200):

                          if os.path.splitext(tar_contents[image_count])[1].lower() == '.jpg':
                              tar.extract(tar_contents[image_count], path=extract_dir)
                              image_id = tar_contents[image_count].split("/", 1)[1]
                              image_id = tar_contents[image_count].split(".", 1)[0]
                              image_id = image_id.split("/", 1)[1]
                        
                              with open(filename, "r") as csvfile:
                                  reader = csv.DictReader(csvfile)

                                  aux = 0
                                  for row in reader:
                                      if (row[column_id] == image_id):
                                          aux = 1
                                          break
                                          
                                  csvfile.seek(0)
                                  if aux == 1:
                                      processed_images +=1
                                      argument = 'Images/' + tar_contents[image_count]
                                      model_string, model_name = models[model_count](argument, temperatures[temp_count], beam_widths[beam_count])


                                      sim_average = 0
                                      rows = []
                                      row_count = 0
                                      for row in reader:
                                          if (row[column_id] == image_id):
                                              rows.append(row[column_sum])
                                      csvfile.seek(0)

                                      excluded_rows = []
                                      j = 0
                                      while (j < len(rows)):
                                          i = 0
                                          sim_average = 0
                                          while (i < len(rows)):
                                              if (i != j):
                                                  embeddings = embed((rows[j], rows[i]))
                                                  sim = cosine_similarity([embeddings[0]], [embeddings[1]])
                                                  #print(sim)
                                                  sim_average += sim
                                              i += 1
                                          sim_average = sim_average/i
                                          if (sim_average < similarity_treshold):
                                              excluded_rows.append(rows[j])
                                          j += 1
                                      #if all descriptions are excluded
                                      if np.shape(excluded_rows) == len(rows) :
                                        excluded_rows[0] = ""

                                      # Iterate over each row in the CSV file and check the value in the specified column
                                      annotations_count = 0
                                      sim_average = 0
                                      for row in reader:
                                          k = 0
                                          aux = 0
                                          while (k < len(excluded_rows)):
                                              if (row[column_sum] == excluded_rows[k]):
                                                  aux = 1
                                              k += 1


                                          if (row[column_id] == image_id and aux == 0):
                                              cell_value = row[column_sum]
                                              #print(cell_value)
                                              s2words_string = cell_value
                                              sentences = [model_string, s2words_string]
                                              embeddings = embed(sentences)

                                              # Compute cosine similarity between the first two sentences
                                              sim = cosine_similarity([embeddings[0]], [embeddings[1]])
                                              annotations_count += 1

                                              sim_average += sim

                                      if annotations_count != 0:
                                          sim_average = sim_average/annotations_count
                                          temp_sim_average += sim_average
                                          sim_average = str(sim_average)
                                          sim_average = sim_average.replace("[","")
                                          sim_average = sim_average.replace("]","")
                                          print(processed_images, "(",image_count+1,")", "\n", model_name, image_id, model_string, sim)
                                          writer.writerow([model_name,image_id,model_string, sim_average, temperatures[temp_count], beam_widths[beam_count]])

                                  
                                  
                                  

                              os.remove('Images/' + tar_contents[image_count])
                          sim_average = 0
                          image_count += 1
                      beam_count += 1
                  temp_sim_average = temp_sim_average / 2104 #processed_images
                  writer.writerow([model_name,"all", temp_sim_average, temperatures[temp_count], "1"])
                  writer_sum.writerow([model_name,temperatures[temp_count],temp_sim_average])
                  temp_count +=1
              model_count += 1

# Close the .tar.gz file
tar.close()

In [ ]:
#Testing beams (2200 images)
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import csv
from PIL import Image
import requests
import torch
import pandas as pd
import tarfile
import os
import warnings
warnings.filterwarnings("ignore", message="Using `max_length`'s default")

torch.cuda.empty_cache()

#https://huggingface.co/google/pix2struct-screen2words-base#using-the-model
def pix2struct_half_gpu(image, temperature, beam_width): #Trained with screen2words(half precision)
    image = Image.open(image)
    
    from transformers import Pix2StructForConditionalGeneration, Pix2StructProcessor
    model = Pix2StructForConditionalGeneration.from_pretrained("google/pix2struct-screen2words-base", torch_dtype=torch.bfloat16).to("cuda")
    processor = Pix2StructProcessor.from_pretrained("google/pix2struct-screen2words-base")

    # image only
    inputs = processor(images=image, text="Mobile application screenshot", return_tensors="pt").to("cuda", torch.bfloat16)

    predictions = model.generate(**inputs, temperature = temperature, num_beams = beam_width, do_sample=True, num_return_sequences=5)
    #print(processor.decode(predictions[0], skip_special_tokens=True))

    model_name = "pix2struct_half_gpu"
    
    pred_step = []

    pred_step.append( str(processor.decode(predictions[0], skip_special_tokens=True)) )
    pred_step.append( str(processor.decode(predictions[1], skip_special_tokens=True)) )
    pred_step.append( str(processor.decode(predictions[2], skip_special_tokens=True)) )
    pred_step.append( str(processor.decode(predictions[3], skip_special_tokens=True)) )
    pred_step.append( str(processor.decode(predictions[4], skip_special_tokens=True)) )

  
    similarity_treshold = 0.6
    j = 0
    aux = 0
    sim_max = -1
    while (j < len(pred_step)):
        i = 0
        sim_average = 0
        while (i < len(pred_step)):
            if (i != j):
                embeddings = embed((pred_step[j], pred_step[i]))
                sim = cosine_similarity([embeddings[0]], [embeddings[1]])
                sim_average += sim
            i += 1
        sim_average = sim_average/i
        if sim_average > sim_max:
            sim_max = sim_average
            pred_max = pred_step[j]

        if (sim_average < similarity_treshold):
            pred_step[j] = ""
        else:
            aux = 1
            
        j += 1

    if aux == 0:
        pred_step[0] = pred_max
        pred_step[1] = str(sim_max)
    #print(pred_step)

    return str(pred_step), model_name

#https://huggingface.co/google/pix2struct-screen2words-base#using-the-model
def pix2struct_full_gpu(image, temperature, beam_width): #Trained with screen2words(full precision)

    image = Image.open(image)
    
    # image only
    inputs = processor(images=image, text="Mobile application screenshot", return_tensors="pt").to("cuda")

    predictions = model.generate(**inputs, temperature = temperature, num_beams = beam_width, do_sample=True, num_return_sequences=5)
    #predictions = model.generate(**inputs.to(torch.bfloat16), temperature = temperature, num_beams = beam_width, do_sample=True, num_return_sequences=5)
    #print(processor.decode(predictions[0], skip_special_tokens=True))

    model_name = "pix2struct_full_gpu"
    pred_step = []

    pred_step.append( str(processor.decode(predictions[0], skip_special_tokens=True)) )
    pred_step.append( str(processor.decode(predictions[1], skip_special_tokens=True)) )
    pred_step.append( str(processor.decode(predictions[2], skip_special_tokens=True)) )
    pred_step.append( str(processor.decode(predictions[3], skip_special_tokens=True)) )
    pred_step.append( str(processor.decode(predictions[4], skip_special_tokens=True)) )

  
    similarity_treshold = 0.6
    j = 0
    aux = 0
    sim_max = -1
    while (j < len(pred_step)):
        i = 0
        sim_average = 0
        while (i < len(pred_step)):
            if (i != j):
                embeddings = embed((pred_step[j], pred_step[i]))
                sim = cosine_similarity([embeddings[0]], [embeddings[1]])
                sim_average += sim
            i += 1
        sim_average = sim_average/i
        if sim_average > sim_max:
            sim_max = sim_average
            pred_max = pred_step[j]

        if (sim_average < similarity_treshold):
            pred_step[j] = ""
        else:
            aux = 1
            
        j += 1

    if aux == 0:
        pred_step[0] = pred_max
        pred_step[1] = str(sim_max)
    #print(pred_step)

    return str(pred_step), model_name



#initializing text comparing model and file
filename = "Annotations/screen_summaries.csv"
column_sum = "summary" 
column_id = "screenId"


#images
#images = ["1264"] #"1264", "1304", "3084", "11337", "13434", "15050", "19821", "21141", "23608", "25372", "25675", "27375", "27392", "29876", "31452", "32435", "34025", "35567", "35604", "35858"
image_count = 0

#models
models = [pix2struct_full_gpu] #pix2struct_full_gpu, pix2struct_half_gpu, blip, pix2stuct_full, pix2struct_prompt, bipin, GIT,  nlpconnect, donut
model_count = 0

#temperatures
temperatures = [0.49] #0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,1.25,1.5,  0.31,0.32,0.33,0.34,0.35,0.36,0.37,0.38,0.39 0.41,0.42,0.43,0.44,0.45,0.46,0.47,0.48,0.49,0.52,0.53,0.54,0.55
#beam_width
beam_widths = [10]

# Set the path to the .tar.gz file
tar_gz_path = 'Images/unique_uis.tar.gz'

# Open the .tar.gz file in read mode
tar = tarfile.open(tar_gz_path, 'r:gz')

# List the contents of the .tar.gz file
tar_contents = tar.getnames()

# Extract the contents of the .tar.gz file to a directory
extract_dir = 'Images'

similarity_treshold = 0.3

# Open a file for writing
with open('Results/Results_summary'+str(temperatures[0])+'-'+str(temperatures[-1])+'.csv', mode='w', newline='') as summary_file:
    writer_sum = csv.writer(summary_file)
    writer_sum.writerow(['Model', 'Beam Width', 'Average Similarity'])

    with open('Results/Results'+str(temperatures[0])+'-'+str(temperatures[-1])+'.csv', mode='w', newline='') as file:

        # Create a writer object
        writer = csv.writer(file)

        # Write a row to the CSV file
        writer.writerow(['Model', 'ImageID','Description','Average Similarity', 'Temperature', 'Beam Width'])

        while (model_count < len(models)):
              temp_count = 0
              while (temp_count < len(temperatures)):
                  beam_count = 0
                  temp_sim_average = 0
                  while (beam_count < len(beam_widths)):
                      model_average = 0
                      image_count = 0
                      processed_images = 0
                      while (image_count < len(tar_contents) and processed_images < 2200):

                          if os.path.splitext(tar_contents[image_count])[1].lower() == '.jpg':
                              tar.extract(tar_contents[image_count], path=extract_dir)
                              image_id = tar_contents[image_count].split("/", 1)[1]
                              image_id = tar_contents[image_count].split(".", 1)[0]
                              image_id = image_id.split("/", 1)[1]
                        
                              with open(filename, "r") as csvfile:
                                  reader = csv.DictReader(csvfile)

                                  aux = 0
                                  for row in reader:
                                      if (row[column_id] == image_id):
                                          aux = 1
                                          break
                                          
                                  csvfile.seek(0)
                                  if aux == 1:
                                      processed_images +=1
                                      argument = 'Images/' + tar_contents[image_count]
                                      model_string, model_name = models[model_count](argument, temperatures[temp_count], beam_widths[beam_count])


                                      sim_average = 0
                                      rows = []
                                      row_count = 0
                                      for row in reader:
                                          if (row[column_id] == image_id):
                                              rows.append(row[column_sum])
                                      csvfile.seek(0)

                                      excluded_rows = []
                                      j = 0
                                      while (j < len(rows)):
                                          i = 0
                                          sim_average = 0
                                          while (i < len(rows)):
                                              if (i != j):
                                                  embeddings = embed((rows[j], rows[i]))
                                                  sim = cosine_similarity([embeddings[0]], [embeddings[1]])
                                                  #print(sim)
                                                  sim_average += sim
                                              i += 1
                                          sim_average = sim_average/i
                                          if (sim_average < similarity_treshold):
                                              excluded_rows.append(rows[j])
                                          j += 1
                                      #if all descriptions are excluded
                                      if np.shape(excluded_rows) == len(rows) :
                                        excluded_rows[0] = ""

                                      # Iterate over each row in the CSV file and check the value in the specified column
                                      annotations_count = 0
                                      sim_average = 0
                                      for row in reader:
                                          k = 0
                                          aux = 0
                                          while (k < len(excluded_rows)):
                                              if (row[column_sum] == excluded_rows[k]):
                                                  aux = 1
                                              k += 1


                                          if (row[column_id] == image_id and aux == 0):
                                              cell_value = row[column_sum]
                                              #print(cell_value)
                                              s2words_string = cell_value
                                              sentences = [model_string, s2words_string]
                                              embeddings = embed(sentences)

                                              # Compute cosine similarity between the first two sentences
                                              sim = cosine_similarity([embeddings[0]], [embeddings[1]])
                                              annotations_count += 1

                                              sim_average += sim

                                      if annotations_count != 0:
                                          sim_average = sim_average/annotations_count
                                          temp_sim_average += sim_average
                                          sim_average = str(sim_average)
                                          sim_average = sim_average.replace("[","")
                                          sim_average = sim_average.replace("]","")
                                          print(processed_images, "(",image_count+1,")", "\n", model_name, image_id, model_string, sim)
                                          writer.writerow([model_name,image_id,model_string, sim_average, temperatures[temp_count], beam_widths[beam_count]])

                                  
                                  
                                  

                              os.remove('Images/' + tar_contents[image_count])
                          sim_average = 0
                          image_count += 1
                      temp_sim_average = temp_sim_average / 2104 #processed_images
                      writer.writerow([model_name,"all", temp_sim_average, beam_widths[beam_count], "1"])
                      writer_sum.writerow([model_name,beam_widths[beam_count],temp_sim_average])
                      beam_count += 1

                  temp_count +=1
              model_count += 1

# Close the .tar.gz file
tar.close()